<a href="https://colab.research.google.com/github/allenalvin333/College4/blob/master/Project/R3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Allen Ben Philipose (18BIS0043)**
# CSE4022 - Racism, before and after George Floyd


## Install

In [1]:
pip install preprocessor

  Created wheel for preprocessor: filename=preprocessor-1.1.3-cp36-none-any.whl size=4477 sha256=f49d7e32397f78c4bf831a43fc1d9914163eb2f599308cfb75702d72355d2e6c
  Stored in directory: /root/.cache/pip/wheels/98/c1/a2/21fbcfd80d76576bbf148991a66f00730f541f265c7600000f
Successfully built preprocessor


In [2]:
pip install nltk

In [3]:
pip install plotly=='4.11.0'

     |████████████████████████████████| 13.1MB 325kB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [4]:
pip install chart-studio

     |████████████████████████████████| 71kB 2.0MB/s 


## Initialize

### Setup Google Drive

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')  

Mounted at /content/gdrive


### Import Libraries

In [5]:
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import csv
from itertools import zip_longest
import nltk
nltk.download('stopwords')
nltk.download('punkt')

import plotly
plotly.__version__
import chart_studio
import chart_studio.plotly as py
from chart_studio import tools
import plotly.graph_objs as go
import pandas as pd

from urllib import request 
from nltk.tokenize import word_tokenize
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Variables

In [33]:
city=[]
latitude=[]
longitude=[]

In [88]:
w = 0
gf = []

### Saved Locations

In [35]:
def location_backup():
  city = ["Montgomery", "Juneau", "Phoenix", "Little Rock", "Sacramento", 
          "Denver", "Hartford", "District of Columbia", "Dover", "Tallahassee", 
          "Atlanta", "Honolulu", "Boise", "Springfield", "Indianapolis", 
          "Des Moines", "Topeka", "Frankfort", "Baton Rouge", "Augusta", 
          "Annapolis", "Boston", "Lansing", "St. Paul", "Jackson", "Jefferson City", 
          "Helena", "Lincoln", "Carson City", "Concord", "Trenton", "Santa Fe", 
          "Albany", "Raleigh", "Bismarck", "Columbus", "Oklahoma City", "Salem", 
          "Harrisburg", "San Juan", "Providence", "Columbia", "Pierre", 
          "Nashville", "Austin", "Salt Lake City", "Montpelier", "Richmond", "Olympia", 
          "Charleston", "Madison", "Cheyenne"]    
  latitude = [32.362,58.302,33.448,33.448,34.736,38.555,39.739,41.767,38.913,39.162,30.452,
              33.760,21.309,43.614,39.791,41.591,39.040,38.197,30.458,44.323,38.973,
              42.235,42.733,44.950,32.320,38.573,46.596,40.810,39.161,43.220,40.221,
              35.667,42.659,35.771,48.813,39.962,35.482,44.931,40.269,18.221,41.823,
              34.000,44.368,36.165,30.266,40.754,44.266,37.540,47.042,38.349,43.074,41.145]
  longitude = [-86.279,-134.419,-112.073,-92.331,-121.468,-104.984,-72.677,-77.013,
               -75.526,-84.272,-84.390,-157.826,-116.237,-89.650,-86.147,-93.620,-95.69,
               -84.863,-91.140,-69.765,-76.501,-71.027,-84.546,-93.094,-90.207,-92.189,
               -112.027,-96.675,-119.753,-71.549,-74.756,-105.964,-73.781,-78.638,-100.779,
               -83.000,-97.535,-123.029,-76.875,-66.590,-71.422,-81.035,-100.336,-86.784,
               -97.750,-111.892,-72.571,-77.460,-122.893,-81.633,-89.384,-104.802]     
  return city, latitude, longitude  

### Scraped Locations


In [36]:
def location():
  city = []
  latitude = []
  longitude = []
  temp,temp1 = [],[]

  url1 = "https://people.sc.fsu.edu/~jburkardt/datasets/states/state_capitals_name.txt"
  html1 = request.urlopen(url1).read().decode('utf8')
  temp = re.findall("\"([a-zA-Z\s]+)\"", html1)
  for i in range(len(temp)):
    if i==45:continue
    city.append(temp[i])

  url2 = "https://people.sc.fsu.edu/~jburkardt/datasets/states/state_capitals_ll.txt"
  html2 = request.urlopen(url2).read().decode('utf8')
  longitude = re.findall('(-\d+\.\d{1,2})', html2)
  temp = re.findall('(\d+\.\d{1,2})', html2)
  for i in range(len(temp)):
    if i==90 or i==91:continue
    temp1.append(temp[i])
  for i in range(0, len(temp1),2):
    latitude.append(temp1[i])
  return city, latitude, longitude

## Functions

In [37]:
class TwitterClient(object):
    def __init__(self):
        consumer_key = '0SwavyP2HPKp5eXCCZDBj6UaA'
        consumer_secret = 'RensiTFX0Ep9e1abuqISGHeeeEHdGVS1dMyfxzdZ4ZxLA9V2P7'
        access_token = '409911991-SPWKqUdOTYYiKvOV52wGiTiGx7gFuWpNd8S6a9Pc'
        access_token_secret = '2Mg0tt12J4aUlh9cxi1gRXhC2zHGR2icNbjpv005A2Z9l'
        try:
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            self.auth.set_access_token(access_token, access_token_secret)
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")

### Cleaning

In [38]:
class TwitterClient(TwitterClient, object):
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

### Sentiment Detection 

In [39]:
class TwitterClient(TwitterClient, object):
    def get_tweet_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        if analysis.sentiment.polarity > 0:
            return 'P'
        elif analysis.sentiment.polarity == 0:
            return 'Neutral'
        else:
            return 'N'

### Get tweets


In [42]:
class TwitterClient(TwitterClient, object):
  def get(self, query, count, date):
    tweets = [] 
    positive = []
    negative = []
    try:
        try:
          city, latitude, longitude = location()
        except:
          city, latitude, longitude = location_backup()
        for i in range(52):
            fetched_tweets = self.api.search(q = query, count = count, since_id = date, coordinates=[latitude[i], longitude[i], 25])
            global w
            for tweet in fetched_tweets:
                parsed_tweet = {}
                parsed_tweet['text'] = tweet.text
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
                if tweet.retweet_count > 0:
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)

            ptweets = [t['text'] for t in tweets if t['sentiment'] == 'P']
            x=(100*len(ptweets)/len(tweets))
            gf.append(x)
            positive.append(ptweets)
            ntweets = [t['text'] for t in tweets if t['sentiment'] == 'N']
            y=(100*len(ntweets)/len(tweets))
            gf.append(y)
            negative.append(ntweets)
            z=(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets))
            gf.append(z)

    except tweepy.TweepError as e:
        print("Error : " + str(e))
    return positive, negative

## Main

In [89]:
api1 = TwitterClient()
p1,n1 = api1.get(query = 'Black Lives Matter', count = 300, date = '202005250000')
p2,n2 = api1.get(query = 'George Floyd', count = 300, date = '202005250000')
p3,n3 = api1.get(query = 'Racism', count = 300, date = '200805250000, 201605250000')

## Processing After

In [90]:
len(gf)

468

In [45]:
mkdir files

### Recording Positives



In [91]:
pos=[] # BLM
for i in range(0,156,3):
    pos.append(gf[i])
city, latitude, longitude = location()
d = [city, latitude, longitude, pos]
export_data = zip_longest(*d, fillvalue = '')
with open('files/1.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("City", "Latitude", "Longitude", "Positive"))
      wr.writerows(export_data)
myfile.close()

In [92]:
pos=[] # George Floyd
for i in range(156,312,3):
    pos.append(gf[i])
city, latitude, longitude = location()
d = [city, latitude, longitude, pos]
export_data = zip_longest(*d, fillvalue = '')
with open('files/3.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("City", "Latitude", "Longitude", "Positive"))
      wr.writerows(export_data)
myfile.close()

In [93]:
pos=[] # Racism before George Floyd
for i in range(312,468,3):
    pos.append(gf[i])
city, latitude, longitude = location()
d = [city, latitude, longitude, pos]
export_data = zip_longest(*d, fillvalue = '')
with open('files/5.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("City", "Latitude", "Longitude", "Positive"))
      wr.writerows(export_data)
myfile.close()

### Recording Negatives

In [94]:
neg=[] # BLM
for i in range(1,156,3):
    neg.append(gf[i])
city, latitude, longitude = location()
d = [city, latitude, longitude, neg]
export_data = zip_longest(*d, fillvalue = '')
with open('files/2.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("City", "Latitude", "Longitude", "Negative"))
      wr.writerows(export_data)
myfile.close()

In [95]:
neg=[] # George Floyd
for i in range(157,312,3):
    neg.append(gf[i])
city, latitude, longitude = location()
d = [city, latitude, longitude, neg]
export_data = zip_longest(*d, fillvalue = '')
with open('files/4.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("City", "Latitude", "Longitude", "Negative"))
      wr.writerows(export_data)
myfile.close()

In [96]:
neg=[] # Racism before George Floyd
for i in range(313,468,3):
    neg.append(gf[i])
city, latitude, longitude = location()
d = [city, latitude, longitude, neg]
export_data = zip_longest(*d, fillvalue = '')
with open('files/6.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(("City", "Latitude", "Longitude", "Negative"))
      wr.writerows(export_data)
myfile.close()

### Black Lives Matter


In [97]:
from chart_studio.tools import set_config_file
from chart_studio.tools import set_credentials_file
set_config_file(
    plotly_domain="https://plotly.com", 
    plotly_api_domain="https://api.plotly.com",
)
set_credentials_file(
    username='allenalvin333', 
    api_key='tfCNZwZe1gQmb3XgwaNe'
)

df = pd.read_csv('files/1.csv')
df.head()

df['text'] = df['City'] + '<br>Positive Sentiment : ' + (df['Positive']).astype(str)+'%'
limits = [(0,100)]
colors = ["rgb(222,50,126)"]
cities = []
scale = 5000

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = go.Scattergeo(
        locationmode = 'ISO-3',
        lon = df_sub['Longitude'],
        lat = df_sub['Latitude'],
        text = df_sub['text'],
        marker = go.scattergeo.Marker(
            size = df_sub['Positive']*50000/scale,
            color = colors[i],
            line = go.scattergeo.marker.Line(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = go.Layout(
        title = go.layout.Title(
            text = 'City-wise Positive sentiments for Black Lives Matter'
        ),
        showlegend = True,
        geo = go.layout.Geo(
            scope = 'north america',
            projection = go.layout.geo.Projection(
                type='equirectangular'
            ),
            showland = True,
            landcolor = 'rgb(0, 0, 0)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        )
    )

fig = go.Figure(data=cities, layout=layout)
py.iplot(fig, filename='Black Lives Matter - Positive')

In [98]:
from chart_studio.tools import set_config_file
from chart_studio.tools import set_credentials_file
set_config_file(
    plotly_domain="https://plotly.com", 
    plotly_api_domain="https://api.plotly.com",
)
set_credentials_file(
    username='allenalvin333', 
    api_key='tfCNZwZe1gQmb3XgwaNe'
)

df = pd.read_csv('files/2.csv')
df.head()

df['text'] = df['City'] + '<br>Negative Sentiment : ' + (df['Negative']).astype(str)+'%'
limits = [(0,100)]
colors = ["rgb(52,225,233)"]
cities = []
scale = 5000

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = go.Scattergeo(
        locationmode = 'ISO-3',
        lon = df_sub['Longitude'],
        lat = df_sub['Latitude'],
        text = df_sub['text'],
        marker = go.scattergeo.Marker(
            size = df_sub['Negative']*50000/scale,
            color = colors[i],
            line = go.scattergeo.marker.Line(
                width=0.5, color='rgb(40,40,40)'
            ),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = go.Layout(
        title = go.layout.Title(
            text = 'City-wise Negative sentiments for Black Lives Matter'
        ),
        showlegend = True,
        geo = go.layout.Geo(
            scope = 'north america',
            projection = go.layout.geo.Projection(
                type='equirectangular'
            ),
            showland = True,
            landcolor = 'rgb(0, 0, 0)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        )
    )

fig = go.Figure(data=cities, layout=layout)
py.iplot(fig, filename='Black Lives Matter - Negative')

### George Floyd


In [99]:
from chart_studio.tools import set_config_file
from chart_studio.tools import set_credentials_file
set_config_file(
    plotly_domain="https://plotly.com", 
    plotly_api_domain="https://api.plotly.com",
)
set_credentials_file(
    username='allenalvin333', 
    api_key='tfCNZwZe1gQmb3XgwaNe'
)

df = pd.read_csv('files/3.csv')
df.head()

df['text'] = df['City'] + '<br>Positive Sentiment : ' + (df['Positive']).astype(str)+'%'
limits = [(0,100)]
colors = ["rgb(222,50,126)"]
cities = []
scale = 5000

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = go.Scattergeo(
        locationmode = 'ISO-3',
        lon = df_sub['Longitude'],
        lat = df_sub['Latitude'],
        text = df_sub['text'],
        marker = go.scattergeo.Marker(
            size = df_sub['Positive']*50000/scale,
            color = colors[i],
            line = go.scattergeo.marker.Line(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = go.Layout(
        title = go.layout.Title(
            text = 'City-wise Positive sentiments for George Floyd'
        ),
        showlegend = True,
        geo = go.layout.Geo(
            scope = 'north america',
            projection = go.layout.geo.Projection(
                type='equirectangular'
            ),
            showland = True,
            landcolor = 'rgb(0, 0, 0)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        )
    )

fig = go.Figure(data=cities, layout=layout)
py.iplot(fig, filename='George Floyd - Positive')

In [100]:
from chart_studio.tools import set_config_file
from chart_studio.tools import set_credentials_file
set_config_file(
    plotly_domain="https://plotly.com", 
    plotly_api_domain="https://api.plotly.com",
)
set_credentials_file(
    username='allenalvin333', 
    api_key='tfCNZwZe1gQmb3XgwaNe'
)

df = pd.read_csv('files/4.csv')
df.head()

df['text'] = df['City'] + '<br>Negative Sentiment : ' + (df['Negative']).astype(str)+'%'
limits = [(0,100)]
colors = ["rgb(52,225,233)"]
cities = []
scale = 5000

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = go.Scattergeo(
        locationmode = 'ISO-3',
        lon = df_sub['Longitude'],
        lat = df_sub['Latitude'],
        text = df_sub['text'],
        marker = go.scattergeo.Marker(
            size = df_sub['Negative']*50000/scale,
            color = colors[i],
            line = go.scattergeo.marker.Line(
                width=0.5, color='rgb(40,40,40)'
            ),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = go.Layout(
        title = go.layout.Title(
            text = 'City-wise Negative sentiments for George Floyd'
        ),
        showlegend = True,
        geo = go.layout.Geo(
            scope = 'north america',
            projection = go.layout.geo.Projection(
                type='equirectangular'
            ),
            showland = True,
            landcolor = 'rgb(0, 0, 0)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        )
    )

fig = go.Figure(data=cities, layout=layout)
py.iplot(fig, filename='George Floyd - Negative')

## Racism before

In [102]:
from chart_studio.tools import set_config_file
from chart_studio.tools import set_credentials_file
set_config_file(
    plotly_domain="https://plotly.com", 
    plotly_api_domain="https://api.plotly.com",
)
set_credentials_file(
    username='allenalvin333', 
    api_key='tfCNZwZe1gQmb3XgwaNe'
)

df = pd.read_csv('files/5.csv')
df.head()

df['text'] = df['City'] + '<br>Positive Sentiment : ' + (df['Positive']).astype(str)+'%'
limits = [(0,100)]
colors = ["rgb(222,50,126)"]
cities = []
scale = 5000

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = go.Scattergeo(
        locationmode = 'ISO-3',
        lon = df_sub['Longitude'],
        lat = df_sub['Latitude'],
        text = df_sub['text'],
        marker = go.scattergeo.Marker(
            size = df_sub['Positive']*50000/scale,
            color = colors[i],
            line = go.scattergeo.marker.Line(width=0.5, color='rgb(40,40,40)'),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = go.Layout(
        title = go.layout.Title(
            text = 'City-wise Positive sentiments for Racism'
        ),
        showlegend = True,
        geo = go.layout.Geo(
            scope = 'north america',
            projection = go.layout.geo.Projection(
                type='equirectangular'
            ),
            showland = True,
            landcolor = 'rgb(0, 0, 0)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        )
    )

fig = go.Figure(data=cities, layout=layout)
py.iplot(fig, filename='Racism - Positive')

In [101]:
for s in p3[0]:
  print(s)
  # print(re.sub("(RT)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", s))

@ChloeAd35418085 @EdanClay Racism is ALIVE AND WELL IN AMERICA AND IT'S NOT GOING ANYWHERE. I THOUGHT POSSIBLY PROG… https://t.co/gBQrSP1w3g
RT @snitchery: look at that white youth vote. now stop peddling the myth that racism is going to magically disappear and die out generation…
Don’t compare the BLM protests and riots to what’s going on at the moment lol BLM riots were over injustice and sys… https://t.co/GPlp2t31nr
RT @moodyredhead: @AndyVonasch Good first step: stop assuming it's racism.

Good second step: actually ASK instead of making assumptions. A…
RT @mgyikes: So if someone voted for trump based on their own personal belief in racism, homophobia, etc. you love them? 🤡
@HalenHg As much as I like it here in SA, it's kind of ignorant to say that bc we're diverse, racism is hardly expe… https://t.co/3rGMWbygG1
yall should watch anne with an e, the amount of racism and every other issues in that series are fkin triggering me it’s an excellent series
RT @emrazz: Still wild that hal

In [104]:
from chart_studio.tools import set_config_file
from chart_studio.tools import set_credentials_file
set_config_file(
    plotly_domain="https://plotly.com", 
    plotly_api_domain="https://api.plotly.com",
)
set_credentials_file(
    username='allenalvin333', 
    api_key='tfCNZwZe1gQmb3XgwaNe'
)

df = pd.read_csv('files/6.csv')
df.head()

df['text'] = df['City'] + '<br>Negative Sentiment : ' + (df['Negative']).astype(str)+'%'
limits = [(0,100)]
colors = ["rgb(52,225,233)"]
cities = []
scale = 5000

for i in range(len(limits)):
    lim = limits[i]
    df_sub = df[lim[0]:lim[1]]
    city = go.Scattergeo(
        locationmode = 'ISO-3',
        lon = df_sub['Longitude'],
        lat = df_sub['Latitude'],
        text = df_sub['text'],
        marker = go.scattergeo.Marker(
            size = df_sub['Negative']*50000/scale,
            color = colors[i],
            line = go.scattergeo.marker.Line(
                width=0.5, color='rgb(40,40,40)'
            ),
            sizemode = 'area'
        ),
        name = '{0} - {1}'.format(lim[0],lim[1]) )
    cities.append(city)

layout = go.Layout(
        title = go.layout.Title(
            text = 'City-wise Negative sentiments for Racism'
        ),
        showlegend = True,
        geo = go.layout.Geo(
            scope = 'north america',
            projection = go.layout.geo.Projection(
                type='equirectangular'
            ),
            showland = True,
            landcolor = 'rgb(0, 0, 0)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        )
    )

fig = go.Figure(data=cities, layout=layout)
py.iplot(fig, filename='Racism - Negative')

In [103]:
for s in n3[0]:
  print(s)
  # print(re.sub("(RT)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", s))

RT @TheAmitie: Ben Franklin, inventor of racism, died on his chamber pot. His last words were: I hate Philadelphia and hope no one thinks I…
@CountDankulaTV Not the same but racism/hate speech is a crime. I find it odd that you’d stick up for either. Both… https://t.co/XqoDIvbg1e
RT @coreylovesyou2: if u voted for trump, block me now. i dont want anything to do with u, i have zero respect for u and i mean this in the…
RT @Drebae_: I hate the fact that they water down racism in the 1900s to bus seats &amp; separate water fountains. They was really kiIIIing us…
The same people who need to hear the word kaffir to recognize racism? They don’t even know politics, just narrow in… https://t.co/hdxllP2kXZ
RT @spideygcv: Remember to call out racism whenever you see it. In real life, online, everywhere. It is never okay.
RT @kashanacauley: Gotta say I’m shocked that an election system developed by slaveowners is so bad at dealing with racism.
RT @EdanClay: I knew racism still existed in America.

## Save Data

In [105]:
!cp -r /content/files /content/gdrive/My\ Drive/Academic/NLP/